## Part 0: Data Loading, Preprocessing, and Visualization

In [ ]:

# %% Setup: Imports and Global Parameters
import os
import cv2 # OpenCV for image manipulation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# tensorflow.keras.utils.to_categorical for one-hot encoding, can be added later if needed for NNs
import random
import pandas as pd
from sklearn.manifold import TSNE # For t-SNE visualization
from sklearn.preprocessing import StandardScaler # For t-SNE and feature scaling
import time # For retrying image reads
from PIL import Image # For robust image opening and format checks

# --- USER CONFIGURABLE PARAMETERS ---
# !!! IMPORTANT: SET THIS TO YOUR DATASET PATH !!!
DATASET_BASE_DIR = 'Birds_25'  # Path to your 'Birds_25' directory

# Image dimensions for resizing and storing in memory (BGR format)
IMG_WIDTH = 128 #
IMG_HEIGHT = 128 #
IMG_CHANNELS = 3 # Images will be stored as BGR

NUM_CLASSES = 25 # As per the assignment
# --- END USER CONFIGURABLE PARAMETERS ---

TRAIN_DIR = os.path.join(DATASET_BASE_DIR, 'train') #
VALID_DIR = os.path.join(DATASET_BASE_DIR, 'valid') #

print(f"Image dimensions for in-memory storage: {IMG_WIDTH}x{IMG_HEIGHT} (BGR)")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Training directory: {TRAIN_DIR}")
print(f"Validation directory (original): {VALID_DIR}")


In [ ]:
species_list = []
if os.path.exists(TRAIN_DIR):
    species_list = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]) #
else:
    print(f"ERROR: Training directory not found at '{TRAIN_DIR}'. Please check DATASET_BASE_DIR.")
    raise FileNotFoundError(f"Training directory not found: {TRAIN_DIR}")

if not species_list:
    print("ERROR: Species list is empty. Ensure dataset is structured correctly.")
else:
    print(f"Found {len(species_list)} species. First 5: {species_list[:5]}...") #
    if len(species_list) != NUM_CLASSES:
        print(f"Warning: Discovered {len(species_list)} species, but NUM_CLASSES is set to {NUM_CLASSES}. Will use discovered count: {len(species_list)}")
        NUM_CLASSES = len(species_list)

all_original_train_paths = [] #
all_original_train_labels_str = [] #
all_original_valid_paths = [] #
all_original_valid_labels_str = [] #

for species_name in species_list:
    species_train_dir = os.path.join(TRAIN_DIR, species_name) #
    if os.path.isdir(species_train_dir):
        for img_file in os.listdir(species_train_dir): #
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')): # Added more common extensions
                all_original_train_paths.append(os.path.join(species_train_dir, img_file)) #
                all_original_train_labels_str.append(species_name) #

    species_valid_dir = os.path.join(VALID_DIR, species_name) #
    if os.path.isdir(species_valid_dir):
        for img_file in os.listdir(species_valid_dir): #
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')): #
                all_original_valid_paths.append(os.path.join(species_valid_dir, img_file)) #
                all_original_valid_labels_str.append(species_name) #

print(f"\nTotal original training image paths collected: {len(all_original_train_paths)}") #
print(f"Total original validation image paths collected: {len(all_original_valid_paths)}") #


In [ ]:
# %% [markdown]
# This step prepares the lists of paths and numerically encoded labels for each dataset split.
# %% Step 2 Code
if not all_original_train_labels_str:
    print("ERROR: No training labels found from Step 1. Cannot proceed with label encoding.")
    # Handle error or ensure Step 1 ran correctly and found images.
    label_encoder = LabelEncoder() # Initialize to prevent later errors, but it won't be fit.
    label_mapping = {}
    all_original_train_labels_encoded = np.array([])
    all_original_valid_labels_encoded = np.array([])
else:
    label_encoder = LabelEncoder() #
    all_original_train_labels_encoded = label_encoder.fit_transform(all_original_train_labels_str) #
    if all_original_valid_labels_str: # Only transform if validation labels exist
         all_original_valid_labels_encoded = label_encoder.transform(all_original_valid_labels_str) #
    else:
        all_original_valid_labels_encoded = np.array([], dtype=int) # Ensure it's an empty array of appropriate type
        print("Warning: No original validation labels found to encode.")

    label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)} #
    print("\nLabel mapping (numerical_label: species_name):") #
    for i in range(min(5, len(label_mapping))): # Print first 5
        print(f"{i}: {label_mapping[i]}")
    if len(label_mapping) > 5: print("...") #
    
    # Update NUM_CLASSES if label_encoder found a different number of classes than initially set
    if len(label_encoder.classes_) != NUM_CLASSES and len(label_encoder.classes_) > 0:
        print(f"Warning: Number of classes from LabelEncoder ({len(label_encoder.classes_)}) differs from NUM_CLASSES ({NUM_CLASSES}). Updating NUM_CLASSES to {len(label_encoder.classes_)}.")
        NUM_CLASSES = len(label_encoder.classes_)
    elif len(label_encoder.classes_) == 0 :
        print("ERROR: LabelEncoder found 0 classes. Dataset might be empty or incorrectly structured.")
        NUM_CLASSES = 0

# --- DEĞİŞİKLİKLER BURADA BAŞLIYOR ---
# Define the number of samples per class for each set
SAMPLES_PER_CLASS_TRAIN = 400
SAMPLES_PER_CLASS_TEST = 80
SAMPLES_PER_CLASS_VALID = 80

# Training set paths and labels
X_train_paths_temp = []
y_train_labels_encoded_list_temp = []
original_train_paths_np = np.array(all_original_train_paths)
original_train_labels_np = np.array(all_original_train_labels_encoded)

for class_idx in range(NUM_CLASSES): #
    class_paths = original_train_paths_np[original_train_labels_np == class_idx]
    if len(class_paths) == 0:
        print(f"Warning: No original training images found for class index {class_idx} ({label_mapping.get(class_idx, 'Unknown')}).")
        continue
    
    random.shuffle(class_paths) # Shuffle paths for this class
    
    selected_train_paths = class_paths[:SAMPLES_PER_CLASS_TRAIN]
    X_train_paths_temp.extend(selected_train_paths)
    y_train_labels_encoded_list_temp.extend([class_idx] * len(selected_train_paths))
    if len(selected_train_paths) < SAMPLES_PER_CLASS_TRAIN:
        print(f"Warning: For training class {label_mapping.get(class_idx, 'Unknown')}, only {len(selected_train_paths)} samples found (requested {SAMPLES_PER_CLASS_TRAIN}).")

X_train_paths = X_train_paths_temp
y_train_labels_encoded_np = np.array(y_train_labels_encoded_list_temp)


# Validation and Test set paths and labels
X_val_paths_temp = []
y_val_labels_encoded_list_temp = []
X_test_paths_temp = []
y_test_labels_encoded_list_temp = []

if all_original_valid_paths: # Proceed only if there are validation paths
    original_valid_paths_np = np.array(all_original_valid_paths) #
    original_valid_labels_np = np.array(all_original_valid_labels_encoded) #

    for class_idx in range(NUM_CLASSES): # Iterate up to the effective NUM_CLASSES
        class_paths = original_valid_paths_np[original_valid_labels_np == class_idx] #
        
        if len(class_paths) < SAMPLES_PER_CLASS_TEST + SAMPLES_PER_CLASS_VALID:
            print(f"Warning: Not enough original validation images for class {label_mapping.get(class_idx, 'Unknown')} to create test ({SAMPLES_PER_CLASS_TEST}) and validation ({SAMPLES_PER_CLASS_VALID}) sets. Found {len(class_paths)}.")
            # Adjust if needed, e.g., by taking fewer or splitting what's available
            # For simplicity here, we'll take what we can, prioritizing test then validation
            random.shuffle(class_paths) #
            current_class_test_paths = class_paths[:SAMPLES_PER_CLASS_TEST]
            current_class_val_paths = class_paths[SAMPLES_PER_CLASS_TEST : SAMPLES_PER_CLASS_TEST + SAMPLES_PER_CLASS_VALID]
            
            X_test_paths_temp.extend(current_class_test_paths)
            y_test_labels_encoded_list_temp.extend([class_idx] * len(current_class_test_paths))
            
            X_val_paths_temp.extend(current_class_val_paths)
            y_val_labels_encoded_list_temp.extend([class_idx] * len(current_class_val_paths))
            continue

        random.shuffle(class_paths) # Shuffle paths for this class
        
        # Assign SAMPLES_PER_CLASS_TEST for test set
        selected_test_paths = class_paths[:SAMPLES_PER_CLASS_TEST]
        X_test_paths_temp.extend(selected_test_paths)
        y_test_labels_encoded_list_temp.extend([class_idx] * len(selected_test_paths))
        
        # Assign SAMPLES_PER_CLASS_VALID for validation set from the remaining
        selected_val_paths = class_paths[SAMPLES_PER_CLASS_TEST : SAMPLES_PER_CLASS_TEST + SAMPLES_PER_CLASS_VALID]
        X_val_paths_temp.extend(selected_val_paths)
        y_val_labels_encoded_list_temp.extend([class_idx] * len(selected_val_paths))
else:
    print("Warning: 'all_original_valid_paths' is empty. Validation and Test sets will be empty.") #

X_val_paths = X_val_paths_temp
y_val_labels_encoded_np = np.array(y_val_labels_encoded_list_temp)
X_test_paths = X_test_paths_temp
y_test_labels_encoded_np = np.array(y_test_labels_encoded_list_temp)

# --- DEĞİŞİKLİKLER BURADA BİTİYOR ---

print(f"\n--- Dataset Split Path Counts (Targeting {SAMPLES_PER_CLASS_TRAIN} Train, {SAMPLES_PER_CLASS_TEST} Test, {SAMPLES_PER_CLASS_VALID} Val per class) ---") #
print(f"Actual training image paths collected: {len(X_train_paths)}") #
print(f"Actual test image paths collected: {len(X_test_paths)}") #
print(f"Actual validation image paths collected: {len(X_val_paths)}") #

# Convert label lists to NumPy arrays (ensure they are arrays even if empty)
y_train_labels_encoded_np = np.array(y_train_labels_encoded_np) #
y_val_labels_encoded_np = np.array(y_val_labels_encoded_list_temp) # Use temp list before this line
y_test_labels_encoded_np = np.array(y_test_labels_encoded_list_temp)# Use temp list before this line

### Part 0.4: Data Preprocessing (Image Loading, Resizing, and Storage in Memory)

In [ ]:

# %% [markdown]
# ## Step 3: Efficient Image Loading, Resizing, and Storage (BGR format in Memory)
# This step reads all images from the split paths ONCE, resizes them, 
# and stores them in memory as BGR NumPy arrays. 
# It also filters labels for images that couldn't be loaded/processed robustly.
# Normalization (e.g., to [0,1]) will be applied later if a specific model requires it.
# For traditional feature extractors (Part 1), we'll often use the 0-255 BGR or Grayscale images derived from these.

# %% Step 3 Code
print("--- Loading and Resizing All Images (BGR format) into Memory & Filtering Labels ---")

def load_resize_and_filter_bgr_efficiently(image_paths, original_labels_np, target_width, target_height, max_retries=3, retry_delay_seconds=1):
    """
    Loads images from paths, resizes to target_width x target_height, stores as BGR.
    Retries reading an image if it fails, up to max_retries using PIL for robustness.
    Filters out images that cannot be loaded/processed and their corresponding labels.
    Returns NumPy arrays of loaded BGR images, filtered labels, and successfully loaded paths.
    """
    loaded_images_bgr_list = []
    filtered_labels_list = []
    successfully_loaded_paths_list = []
    skipped_count = 0
    
    if not image_paths: # Handle empty image_paths list
        print("Warning: Input image_paths list is empty for efficient loading.")
        # Return empty arrays with appropriate shapes if possible, or just empty arrays
        return np.empty((0, target_height, target_width, IMG_CHANNELS), dtype=np.uint8), \
               np.array([], dtype=original_labels_np.dtype if original_labels_np.size > 0 else int), \
               []

    total_paths = len(image_paths)
    print(f"Attempting to load and resize {total_paths} images to ({target_width}x{target_height})...")
    
    for i, img_path in enumerate(image_paths):
        img_bgr = None
        for attempt in range(max_retries):
            try:
                pil_img = Image.open(img_path)
                pil_img_rgb = pil_img.convert('RGB') 
                img_bgr = cv2.cvtColor(np.array(pil_img_rgb), cv2.COLOR_RGB2BGR)
                
                if img_bgr is not None:
                    break 
            except FileNotFoundError:
                print(f"ERROR (Attempt {attempt+1}/{max_retries}): File not found {img_path}. Skipping this image.")
                img_bgr = None 
                break 
            except Exception as e_read:
                print(f"Warning (Attempt {attempt+1}/{max_retries}): Error reading/converting image {img_path}: {e_read}. Retrying in {retry_delay_seconds}s...")
                time.sleep(retry_delay_seconds)
        
        if img_bgr is None:
            print(f"ERROR: Failed to load/convert image {img_path} after {max_retries} attempts, skipping.")
            skipped_count += 1
            continue
        
        try:
            # Ensure image has 3 channels after conversion
            if len(img_bgr.shape) != 3 or img_bgr.shape[2] != 3:
                print(f"Warning: Image {img_path} does not have 3 channels after conversion (shape: {img_bgr.shape}), attempting to force BGR.")
                if len(img_bgr.shape) == 2: 
                    img_bgr = cv2.cvtColor(img_bgr, cv2.COLOR_GRAY2BGR)
                elif img_bgr.shape[2] == 1: 
                     img_bgr = cv2.cvtColor(img_bgr, cv2.COLOR_GRAY2BGR)
                elif img_bgr.shape[2] == 4: 
                    img_bgr = cv2.cvtColor(img_bgr, cv2.COLOR_BGRA2BGR)
                else:
                    raise ValueError(f"Unsupported number of channels: {img_bgr.shape[2]}")

            img_bgr_resized = cv2.resize(img_bgr, (target_width, target_height), interpolation=cv2.INTER_AREA)
            loaded_images_bgr_list.append(img_bgr_resized)
            filtered_labels_list.append(original_labels_np[i])
            successfully_loaded_paths_list.append(img_path)
        except Exception as e_proc:
            print(f"Error resizing/processing image {img_path} (shape: {img_bgr.shape if img_bgr is not None else 'None'}, dtype: {img_bgr.dtype if img_bgr is not None else 'None'}): {e_proc}, skipping.")
            skipped_count += 1
            continue

        if (i + 1) % 250 == 0 or (i + 1) == total_paths: 
            print(f"  Processed {i+1}/{total_paths} image paths for this set.")
            
    print(f"Finished loading for this set. Successfully loaded/resized {len(loaded_images_bgr_list)} images. Skipped {skipped_count} images.")
    
    # Convert lists to NumPy arrays, ensuring correct dtype and shape for empty lists
    final_images_array = np.array(loaded_images_bgr_list, dtype=np.uint8) if loaded_images_bgr_list else np.empty((0, target_height, target_width, IMG_CHANNELS), dtype=np.uint8)
    final_labels_array = np.array(filtered_labels_list, dtype=original_labels_np.dtype if original_labels_np.size > 0 else int) if filtered_labels_list else np.array([], dtype=original_labels_np.dtype if original_labels_np.size > 0 else int)
    
    return final_images_array, final_labels_array, successfully_loaded_paths_list

# --- Load all images into memory. These will be used by subsequent parts. ---
# The *_final variables will hold the actual BGR image data (0-255 range) and their filtered labels.
# X_train_paths etc. are from Step 2 Code cell

if 'X_train_paths' not in globals() or not X_train_paths:
    print("ERROR: X_train_paths is not defined or empty. Please run Step 1 and Step 2 first.")
    # Initialize to prevent errors if subsequent cells are run accidentally
    X_train_images_bgr, y_train_final, X_train_paths_final = np.array([]), np.array([]), []
    X_val_images_bgr, y_val_final, X_val_paths_final = np.array([]), np.array([]), []
    X_test_images_bgr, y_test_final, X_test_paths_final = np.array([]), np.array([]), []
else:
    X_train_images_bgr, y_train_final, X_train_paths_final = load_resize_and_filter_bgr_efficiently(X_train_paths, y_train_labels_encoded_np, IMG_WIDTH, IMG_HEIGHT)
    X_val_images_bgr, y_val_final, X_val_paths_final = load_resize_and_filter_bgr_efficiently(X_val_paths, y_val_labels_encoded_np, IMG_WIDTH, IMG_HEIGHT)
    X_test_images_bgr, y_test_final, X_test_paths_final = load_resize_and_filter_bgr_efficiently(X_test_paths, y_test_labels_encoded_np, IMG_WIDTH, IMG_HEIGHT)

print("\n--- Final Data Shapes After Loading Images into Memory ---")
print(f"X_train_images_bgr shape: {X_train_images_bgr.shape}, y_train_final shape: {y_train_final.shape}")
print(f"X_val_images_bgr shape: {X_val_images_bgr.shape}, y_val_final shape: {y_val_final.shape}")
print(f"X_test_images_bgr shape: {X_test_images_bgr.shape}, y_test_final shape: {y_test_final.shape}")

# Update NUM_CLASSES and target_names_part1 based on actual unique labels found AFTER filtering
# This is crucial if some classes were entirely skipped due to loading errors.
if y_train_final.size > 0:
    # Concatenate all filtered labels to find the true set of classes present in the loaded data
    all_loaded_labels_list = []
    if y_train_final.size > 0: all_loaded_labels_list.append(y_train_final)
    if y_val_final.size > 0: all_loaded_labels_list.append(y_val_final)
    if y_test_final.size > 0: all_loaded_labels_list.append(y_test_final)
    
    if all_loaded_labels_list: # If any labels exist after filtering
        all_loaded_labels = np.concatenate(all_loaded_labels_list, axis=0)
        unique_loaded_labels = np.unique(all_loaded_labels)
        actual_num_classes_loaded = len(unique_loaded_labels)
    else: # No labels loaded at all
        actual_num_classes_loaded = 0
        unique_loaded_labels = np.array([])
        print("CRITICAL WARNING: No labels loaded into y_train_final, y_val_final, or y_test_final. Dataset might be empty or all images failed to load.")

    if actual_num_classes_loaded != NUM_CLASSES:
        print(f"INFO: Number of unique labels in all loaded data ({actual_num_classes_loaded}) "
              f"differs from initial NUM_CLASSES ({NUM_CLASSES}). Updating NUM_CLASSES to {actual_num_classes_loaded}.")
        NUM_CLASSES = actual_num_classes_loaded
    
    if 'label_mapping' in globals():
        # Create target names based on labels that are actually present and in label_mapping
        target_names_part1 = [label_mapping.get(i, str(i)) for i in sorted(list(unique_loaded_labels))]
        if len(target_names_part1) != actual_num_classes_loaded and actual_num_classes_loaded > 0:
             print(f"Warning: Mismatch in target_names_part1 generation ({len(target_names_part1)}) and actual_num_classes_loaded ({actual_num_classes_loaded}). Some labels might not be in label_mapping. Using sorted unique labels as strings for missing ones.")
             # This line ensures target_names_part1 has the correct length, using string of label if not in mapping
             target_names_part1 = [label_mapping.get(i, str(i)) for i in sorted(list(unique_loaded_labels))]
    else: # Fallback if label_mapping is not defined
        target_names_part1 = [str(i) for i in sorted(list(unique_loaded_labels))]
        if actual_num_classes_loaded > 0 : print("Warning: label_mapping not found. Using sorted unique numerical labels for classification report target names.")
        else: print("Warning: label_mapping not found and no labels loaded to derive target_names.")

    print(f"Effective NUM_CLASSES for reports: {NUM_CLASSES}")
    print(f"Target names for reports (first 5 if available): {target_names_part1[:5] if target_names_part1 else 'N/A'}")

else: # Handle case where y_train_final itself is empty (meaning no training images loaded)
    print("ERROR: y_train_final is empty after loading. Cannot reliably set NUM_CLASSES or target_names_part1. This indicates a major issue with training data loading.")
    NUM_CLASSES = 0 # Set to 0 if no training data, to prevent errors in later cells expecting NUM_CLASSES
    target_names_part1 = []



## Part 1

In [120]:
# %% [markdown]
# ## Part 1: Classification According to Feature Extraction (HSV, HoG, Gabor - Parallel with MLP)

# %%
# Önceki importlarınıza ek olarak veya mevcut olanları kontrol ederek:
from skimage.feature import hog
from skimage import color, filters # Gabor için filters modülü
import cv2 # OpenCV zaten import edilmiş olmalı
import numpy as np
from multiprocessing import Pool, cpu_count
import time

# ML Modelleri ve Pipeline için gerekli importlar
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier # Naive Bayes yerine MLPClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score


In [121]:

# %%
# Part 0'dan gelen değişkenler varsayılıyor:
# X_train_images_bgr, y_train_final
# X_test_images_bgr, y_test_final
# target_names_part1 (sınıf isimleri listesi)
# IMG_WIDTH, IMG_HEIGHT (örneğin 128, 128)

# --- Özellik Çıkarım Fonksiyonları ---
# (extract_hsv_histogram_single_image, extract_hog_features_single_image, extract_gabor_features_single_image fonksiyonları
# bir önceki yanıttaki gibi burada tanımlı olmalıdır)

def extract_hsv_histogram_single_image(image_bgr, h_bins=8, s_bins=4, v_bins=4):
    """Tek bir BGR resimden HSV renk histogramı özelliklerini çıkarır."""
    if image_bgr is None:
        return None
    try:
        image_hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([image_hsv], [0, 1, 2], None, 
                            [h_bins, s_bins, v_bins], 
                            [0, 180, 0, 256, 0, 256]) # Hue için 0-179 aralığı
        cv2.normalize(hist, hist) # Normalizasyon
        return hist.flatten()
    except Exception as e:
        print(f"Error extracting HSV histogram: {e}")
        return None

def extract_hog_features_single_image(image_bgr):
    """Tek bir BGR resimden HoG özelliklerini çıkarır."""
    if image_bgr is None:
        return None
    try:
        image_gray = color.rgb2gray(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
        features = hog(image_gray, pixels_per_cell=(16, 16), 
                       cells_per_block=(2, 2), orientations=9,
                       visualize=False, feature_vector=True, block_norm='L2-Hys')
        return features
    except Exception as e:
        print(f"Error extracting HoG features: {e}")
        return None

def extract_gabor_features_single_image(image_bgr, num_orientations=8, frequencies=(0.05, 0.25, 0.5), sigmas=(1,3)):
    """Tek bir BGR resimden Gabor filtresi tepkilerinin ortalama ve std. sapmasını çıkarır."""
    if image_bgr is None:
        return None
    try:
        image_gray = color.rgb2gray(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
        gabor_features = []
        for theta_idx in range(num_orientations):
            theta = theta_idx / float(num_orientations) * np.pi
            for frequency in frequencies:
                for sigma_val in sigmas:
                    filt_real, filt_imag = filters.gabor(image_gray, frequency=frequency, theta=theta, sigma_x=sigma_val, sigma_y=sigma_val)
                    magnitude = np.sqrt(filt_real**2 + filt_imag**2)
                    gabor_features.append(np.mean(magnitude))
                    gabor_features.append(np.std(magnitude))
        return np.array(gabor_features)
    except Exception as e:
        print(f"Error extracting Gabor features: {e}")
        return None

# %%
# --- ML Modellerini Getiren Fonksiyon (NaiveBayes yerine MLP ile GÜNCELLENDİ) ---
def get_ml_models():
    models = {
        "SVM": SVC(kernel='rbf', C=1.0, random_state=42, probability=True, max_iter=1000),
        "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        "LogisticRegression": LogisticRegression(solver='liblinear', max_iter=200, random_state=42), 
        "MLP": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42, early_stopping=True) # NaiveBayes yerine
    }
    return models

# %%
# --- Her bir işlem (thread/process) için ana işleyici fonksiyon ---
# process_feature_and_models fonksiyonu bir önceki yanıttaki (Turn 13 veya 14) gibi kalacak.
# Sadece get_ml_models() fonksiyonunun güncellenmiş halini kullanacak.
# Bu fonksiyonu bir önceki yanıttan kopyalayıp buraya veya bir önceki hücreye ekleyebilirsiniz.
# Eğer zaten notebook'unuzda varsa, sadece get_ml_models çağrısının doğru olduğundan emin olun.
# (Örnek: Bir önceki yanıttaki gibi process_feature_and_models fonksiyonu)

def process_feature_and_models(feature_name, feature_extractor_func,
                               X_train_imgs, y_train_labels,
                               X_test_imgs, y_test_labels,
                               ml_models_dict, target_names):
    """
    Belirli bir özellik çıkarıcıyı kullanarak özellikleri çıkarır,
    ardından verilen ML modellerini eğitir ve test eder.
    """
    print(f"--- Starting process for {feature_name} ---")
    
    # 1. Özellik Çıkarımı
    print(f"Extracting {feature_name} features for training set...")
    start_fe_time = time.time()
    # Özellik çıkarımını doğrudan numpy array üzerinde yapalım
    X_train_features_list = [feature_extractor_func(img) for img in X_train_imgs]
    
    # Başarısız olanları (None dönenleri) ve karşılık gelen etiketleri filtrele
    successful_train_indices = [i for i, f in enumerate(X_train_features_list) if f is not None]
    if len(successful_train_indices) == 0:
        print(f"ERROR: No training features successfully extracted for {feature_name}. Skipping.")
        return {feature_name: "Feature extraction failed for all training samples."}
    X_train_features = np.array([X_train_features_list[i] for i in successful_train_indices])
    y_train_labels_filtered = y_train_labels[successful_train_indices]

    if X_train_features.size == 0:
        print(f"ERROR: No training features extracted (after filtering None) for {feature_name}. Skipping.")
        return {feature_name: "Feature extraction resulted in empty training set."}

    print(f"Extracting {feature_name} features for test set...")
    X_test_features_list = [feature_extractor_func(img) for img in X_test_imgs]
    
    successful_test_indices = [i for i, f in enumerate(X_test_features_list) if f is not None]
    if len(successful_test_indices) == 0:
        print(f"ERROR: No test features successfully extracted for {feature_name}. Skipping model evaluations for this feature.")
        return {feature_name: "Feature extraction failed for all test samples."}
    X_test_features = np.array([X_test_features_list[i] for i in successful_test_indices])
    y_test_labels_filtered = y_test_labels[successful_test_indices]
    
    if X_test_features.size == 0:
        print(f"ERROR: No test features extracted (after filtering None) for {feature_name}. Skipping.")
        return {feature_name: "Feature extraction resulted in empty test set."}

    end_fe_time = time.time()
    print(f"{feature_name} feature extraction completed in {end_fe_time - start_fe_time:.2f} seconds.")
    print(f"Train features shape: {X_train_features.shape}, Test features shape: {X_test_features.shape}")

    results = {}
    
    # 2. ML Modellerini Eğitme ve Test Etme
    for model_name, model_instance in ml_models_dict.items():
        print(f"\nTraining {model_name} with {feature_name} features...")
        pipeline = Pipeline([
            ('scaler', StandardScaler()), 
            ('classifier', model_instance)
        ])
        
        start_train_time = time.time()
        try:
            if X_train_features.shape[1] != X_test_features.shape[1]:
                 print(f"ERROR: Feature dimension mismatch for {feature_name} between train ({X_train_features.shape[1]}) and test ({X_test_features.shape[1]}). Skipping {model_name}.")
                 results[f"{feature_name}_{model_name}"] = {"error": "Feature dimension mismatch"}
                 continue

            if X_train_features.shape[0] == 0 or y_train_labels_filtered.shape[0] == 0:
                print(f"ERROR: Empty training features or labels for {feature_name}. Skipping {model_name}.")
                results[f"{feature_name}_{model_name}"] = {"error": "Empty training data"}
                continue
            
            if len(np.unique(y_train_labels_filtered)) < 2 :
                print(f"ERROR: Training data for {feature_name} has less than 2 classes. Skipping {model_name}.")
                results[f"{feature_name}_{model_name}"] = {"error": "Less than 2 classes in training data"}
                continue


            pipeline.fit(X_train_features, y_train_labels_filtered)
        except ValueError as e:
            print(f"ERROR training {model_name} with {feature_name}: {e}. Skipping this model.")
            results[f"{feature_name}_{model_name}"] = {"error": str(e)}
            continue
        end_train_time = time.time()
        print(f"{model_name} training completed in {end_train_time - start_train_time:.2f} seconds.")
        
        # Test seti üzerinde değerlendirme
        if X_test_features.shape[0] > 0 and y_test_labels_filtered.shape[0] > 0 :
            if len(np.unique(y_test_labels_filtered)) < 2 and len(np.unique(y_test_labels_filtered)) != len(np.unique(y_train_labels_filtered)):
                 print(f"Warning: Test data for {feature_name} has less than 2 classes or different class set than train. Report might be problematic.")
            
            y_pred = pipeline.predict(X_test_features)
            accuracy = accuracy_score(y_test_labels_filtered, y_pred)
            try:
                # Sınıflandırma raporu için etiketlerin hem train hem de test setinde olmasını sağlamak gerekebilir
                # Eğer target_names verilmiyorsa, sadece mevcut etiketleri kullanır.
                # Modelin eğitildiği sınıflarla testteki sınıflar aynı olmalı.
                unique_labels_in_test_and_train = sorted(list(set(y_train_labels_filtered) | set(y_test_labels_filtered)))
                current_target_names = [target_names[i] for i in unique_labels_in_test_and_train if i < len(target_names)]

                report = classification_report(y_test_labels_filtered, y_pred, labels=unique_labels_in_test_and_train, target_names=current_target_names, zero_division=0, output_dict=True)
            except ValueError as e: 
                print(f"Warning: Could not generate classification report for {model_name} with {feature_name} due to label mismatch or other issues: {e}")
                report = {"error": str(e), "accuracy_manual": accuracy}


            print(f"\n{model_name} with {feature_name} - Test Set:")
            print(f"Accuracy: {accuracy:.4f}")
            if "error" not in report and 'macro avg' in report:
                print(f"Macro Avg F1-score: {report.get('macro avg', {}).get('f1-score', 'N/A'):.4f}")
            
            results[f"{feature_name}_{model_name}"] = {
                "accuracy": accuracy,
                "classification_report_dict": report 
            }
        else:
            print(f"Test features for {feature_name} are empty or labels are missing. Skipping evaluation for {model_name}.")
            results[f"{feature_name}_{model_name}"] = {"error": "Empty test features or labels"}
            
    print(f"--- Finished process for {feature_name} ---")
    return {feature_name: results}





In [ ]:
# %%
import time
from multiprocessing import Pool, cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# %%
# --- Helper function for chunked Gabor processing ---
def extract_gabor_features_chunk(args):
    """
    Extract Gabor features for a chunk of images
    Args: (image_chunk, start_idx, chunk_id, feature_extractor_func)
    Returns: (chunk_id, start_idx, features_list)
    """
    image_chunk, start_idx, chunk_id, feature_extractor_func = args
    print(f"Processing Gabor chunk {chunk_id + 1} (images {start_idx} to {start_idx + len(image_chunk) - 1})")
    
    features_list = []
    for img in image_chunk:
        features = feature_extractor_func(img)
        features_list.append(features)
    
    print(f"Gabor chunk {chunk_id + 1} completed")
    return chunk_id, start_idx, features_list

# %%
# --- Phase 1: Feature Extraction Functions ---
def extract_features_single_core(feature_name, feature_extractor_func, X_images, dataset_type="train"):
    """
    Extract features using single core (for HSV and HoG)
    """
    print(f"Extracting {feature_name} features for {dataset_type} set (single core)...")
    start_time = time.time()
    
    features_list = [feature_extractor_func(img) for img in X_images]
    
    # Filter out failed extractions (None values)
    successful_indices = [i for i, f in enumerate(features_list) if f is not None]
    
    if len(successful_indices) == 0:
        print(f"ERROR: No {dataset_type} features successfully extracted for {feature_name}")
        return feature_name, dataset_type, None, None
    
    features_array = np.array([features_list[i] for i in successful_indices])
    
    end_time = time.time()
    print(f"{feature_name} {dataset_type} feature extraction completed in {end_time - start_time:.2f} seconds.")
    print(f"{dataset_type.capitalize()} features shape: {features_array.shape}")
    
    return feature_name, dataset_type, features_array, successful_indices

def extract_gabor_features_multicore(X_images, dataset_type="train", num_cores=10):
    """
    Extract Gabor features using multiple cores by splitting the dataset
    """
    print(f"Extracting Gabor features for {dataset_type} set using {num_cores} cores...")
    start_time = time.time()
    
    # Split images into chunks for parallel processing
    total_images = len(X_images)
    chunk_size = max(1, total_images // num_cores)
    
    chunks = []
    for i in range(0, total_images, chunk_size):
        end_idx = min(i + chunk_size, total_images)
        image_chunk = X_images[i:end_idx]
        chunks.append((image_chunk, i, len(chunks), extract_gabor_features_single_image))
    
    print(f"Split {total_images} images into {len(chunks)} chunks of ~{chunk_size} images each")
    
    # Process chunks in parallel
    try:
        with Pool(processes=num_cores) as pool:
            chunk_results = pool.map(extract_gabor_features_chunk, chunks)
        
        # Combine results from all chunks
        # Sort by chunk_id to maintain order
        chunk_results.sort(key=lambda x: x[0])
        
        combined_features_list = []
        for chunk_id, start_idx, features_list in chunk_results:
            combined_features_list.extend(features_list)
        
        # Filter out failed extractions (None values)
        successful_indices = [i for i, f in enumerate(combined_features_list) if f is not None]
        
        if len(successful_indices) == 0:
            print(f"ERROR: No {dataset_type} Gabor features successfully extracted")
            return "Gabor", dataset_type, None, None
        
        features_array = np.array([combined_features_list[i] for i in successful_indices])
        
        end_time = time.time()
        print(f"Gabor {dataset_type} feature extraction completed in {end_time - start_time:.2f} seconds.")
        print(f"{dataset_type.capitalize()} features shape: {features_array.shape}")
        
        return "Gabor", dataset_type, features_array, successful_indices
        
    except Exception as e:
        print(f"Error during Gabor feature extraction: {e}")
        return "Gabor", dataset_type, None, None

# %%
# --- Phase 2: Parallel Model Training and Evaluation ---
def train_and_evaluate_model(args):
    """
    Train and evaluate a single model with given features
    Args: (feature_name, model_name, model_instance, X_train_features, y_train_filtered, 
           X_test_features, y_test_filtered, target_names)
    """
    (feature_name, model_name, model_instance, X_train_features, y_train_filtered,
     X_test_features, y_test_filtered, target_names) = args
    
    print(f"Training {model_name} with {feature_name} features...")
    
    # Create pipeline with scaling
    pipeline = Pipeline([
        ('scaler', StandardScaler()), 
        ('classifier', model_instance)
    ])
    
    start_train_time = time.time()
    
    try:
        # Validation checks
        if X_train_features.shape[1] != X_test_features.shape[1]:
            return (feature_name, model_name, {"error": "Feature dimension mismatch"})
        
        if X_train_features.shape[0] == 0 or y_train_filtered.shape[0] == 0:
            return (feature_name, model_name, {"error": "Empty training data"})
        
        if len(np.unique(y_train_filtered)) < 2:
            return (feature_name, model_name, {"error": "Less than 2 classes in training data"})
        
        # Train the model
        pipeline.fit(X_train_features, y_train_filtered)
        
        end_train_time = time.time()
        print(f"{model_name} with {feature_name} training completed in {end_train_time - start_train_time:.2f} seconds.")
        
        # Evaluate on test set
        if X_test_features.shape[0] > 0 and y_test_filtered.shape[0] > 0:
            y_pred = pipeline.predict(X_test_features)
            accuracy = accuracy_score(y_test_filtered, y_pred)
            
            try:
                # Generate classification report
                unique_labels = sorted(list(set(y_train_filtered) | set(y_test_filtered)))
                current_target_names = [target_names[i] for i in unique_labels if i < len(target_names)]
                
                report = classification_report(
                    y_test_filtered, y_pred, 
                    labels=unique_labels, 
                    target_names=current_target_names, 
                    zero_division=0, 
                    output_dict=True
                )
                
                print(f"{model_name} with {feature_name} - Accuracy: {accuracy:.4f}")
                if 'macro avg' in report:
                    print(f"{model_name} with {feature_name} - Macro F1: {report['macro avg']['f1-score']:.4f}")
                
                return (feature_name, model_name, {
                    "accuracy": accuracy,
                    "classification_report_dict": report
                })
                
            except ValueError as e:
                print(f"Warning: Could not generate classification report for {model_name} with {feature_name}: {e}")
                return (feature_name, model_name, {
                    "accuracy": accuracy,
                    "classification_report_dict": {"error": str(e), "accuracy_manual": accuracy}
                })
        else:
            return (feature_name, model_name, {"error": "Empty test features or labels"})
            
    except Exception as e:
        print(f"ERROR training {model_name} with {feature_name}: {e}")
        return (feature_name, model_name, {"error": str(e)})

# %%
# --- Main Execution Code ---
# Feature extractors and ML models
feature_extractors_to_test = {
    "HSV_Hist": extract_hsv_histogram_single_image,
    "HoG": extract_hog_features_single_image,
    "Gabor": extract_gabor_features_single_image
}

ml_models = get_ml_models()  # MLP included

# Check if data is available
if not all(hasattr(arr, 'size') and arr.size > 0 for arr in [X_train_images_bgr, y_train_final, X_test_images_bgr, y_test_final]):
    print("One or more data arrays are empty or not initialized. Skipping Part 1.")
    part1_results = {}
else:
    print(f"Starting optimized parallel processing with {cpu_count()} cores available...")
    print("Core allocation: HSV (1 core), HoG (1 core), Gabor (10 cores)")
    
    # ===== PHASE 1: PARALLEL FEATURE EXTRACTION =====
    print("\n=== PHASE 1: FEATURE EXTRACTION ===")
    
    extracted_features = {}
    successful_indices = {}
    
    try:
        # Start HSV and HoG feature extraction in parallel (2 cores total)
        with ProcessPoolExecutor(max_workers=2) as executor:
            # Submit HSV tasks
            hsv_train_future = executor.submit(
                extract_features_single_core, "HSV_Hist", 
                extract_hsv_histogram_single_image, X_train_images_bgr, "train"
            )
            hsv_test_future = executor.submit(
                extract_features_single_core, "HSV_Hist", 
                extract_hsv_histogram_single_image, X_test_images_bgr, "test"
            )
            
            # Submit HoG tasks
            hog_train_future = executor.submit(
                extract_features_single_core, "HoG", 
                extract_hog_features_single_image, X_train_images_bgr, "train"
            )
            hog_test_future = executor.submit(
                extract_features_single_core, "HoG", 
                extract_hog_features_single_image, X_test_images_bgr, "test"
            )
            
            # Process HSV and HoG results
            hsv_hog_futures = [hsv_train_future, hsv_test_future, hog_train_future, hog_test_future]
            
            for future in as_completed(hsv_hog_futures):
                feat_name, dataset_type, features_array, indices = future.result()
                if features_array is not None:
                    extracted_features[f"{feat_name}_{dataset_type}"] = features_array
                    successful_indices[f"{feat_name}_{dataset_type}"] = indices
                else:
                    print(f"Failed to extract {feat_name} features for {dataset_type} set")
        
        # Extract Gabor features using 10 cores
        print("\nStarting Gabor feature extraction with 10 cores...")
        
        # Extract Gabor train features
        gabor_train_result = extract_gabor_features_multicore(X_train_images_bgr, "train", num_cores=10)
        feat_name, dataset_type, features_array, indices = gabor_train_result
        if features_array is not None:
            extracted_features[f"{feat_name}_{dataset_type}"] = features_array
            successful_indices[f"{feat_name}_{dataset_type}"] = indices
        else:
            print(f"Failed to extract {feat_name} features for {dataset_type} set")
        
        # Extract Gabor test features
        gabor_test_result = extract_gabor_features_multicore(X_test_images_bgr, "test", num_cores=10)
        feat_name, dataset_type, features_array, indices = gabor_test_result
        if features_array is not None:
            extracted_features[f"{feat_name}_{dataset_type}"] = features_array
            successful_indices[f"{feat_name}_{dataset_type}"] = indices
        else:
            print(f"Failed to extract {feat_name} features for {dataset_type} set")
                
    except Exception as e:
        print(f"Error during feature extraction: {e}")
        extracted_features = {}
    
    print(f"\nFeature extraction completed. Extracted features for: {list(extracted_features.keys())}")
    
    # ===== PHASE 2: PARALLEL MODEL TRAINING (12 cores) =====
    print("\n=== PHASE 2: MODEL TRAINING AND EVALUATION ===")
    
    if extracted_features:
        # Prepare training tasks (3 features × 4 models = 12 tasks)
        training_tasks = []
        
        feature_name_mapping = {
            "HSV_Hist": "HSV_Hist",
            "HoG": "HoG", 
            "Gabor": "Gabor"
        }
        
        for original_feat_name, mapped_feat_name in feature_name_mapping.items():
            train_key = f"{mapped_feat_name}_train"
            test_key = f"{mapped_feat_name}_test"
            
            if train_key in extracted_features and test_key in extracted_features:
                # Get filtered labels based on successful feature extractions
                y_train_filtered = y_train_final[successful_indices[train_key]]
                y_test_filtered = y_test_final[successful_indices[test_key]]
                
                # Create training tasks for all models with this feature
                for model_name, model_instance in ml_models.items():
                    task = (
                        mapped_feat_name, model_name, model_instance,
                        extracted_features[train_key], y_train_filtered,
                        extracted_features[test_key], y_test_filtered,
                        target_names_part1
                    )
                    training_tasks.append(task)
        
        print(f"Created {len(training_tasks)} training tasks")
        
        # Execute all training tasks in parallel using all 12 cores
        part1_results = {}
        
        if training_tasks:
            try:
                with Pool(processes=12) as pool:  # Use all 12 cores for model training
                    training_results = pool.map(train_and_evaluate_model, training_tasks)
                
                # Organize results
                for feat_name, model_name, result in training_results:
                    key = f"{feat_name}_{model_name}"
                    part1_results[key] = result
                    
            except Exception as e:
                print(f"Error during model training: {e}")
                part1_results = {}
        else:
            print("No training tasks to execute")
            part1_results = {}
    else:
        print("No features extracted successfully. Skipping model training.")
        part1_results = {}

print("\n=== PART 1: ALL PARALLEL PROCESSES COMPLETED ===")

# %%
# --- Results Summary ---
if part1_results:
    print("\n=== RESULTS SUMMARY ===")
    
    # Sort results by accuracy for better readability
    sorted_results = []
    for key, result in part1_results.items():
        if isinstance(result, dict) and "accuracy" in result:
            sorted_results.append((key, result["accuracy"], result))
        else:
            print(f"{key}: {result}")
    
    # Sort by accuracy (descending)
    sorted_results.sort(key=lambda x: x[1], reverse=True)
    
    print("\nTop performing combinations:")
    for key, accuracy, result in sorted_results[:10]:  # Show top 10
        feature_name, model_name = key.split('_', 1)
        print(f"{feature_name} + {model_name}: {accuracy:.4f}")
        
        if "classification_report_dict" in result and "macro avg" in result["classification_report_dict"]:
            macro_f1 = result["classification_report_dict"]["macro avg"]["f1-score"]
            print(f"  Macro F1-Score: {macro_f1:.4f}")
        print()
    
    # Find best overall combination
    if sorted_results:
        best_combo, best_accuracy, best_result = sorted_results[0]
        best_feature, best_model = best_combo.split('_', 1)
        print(f"🏆 BEST COMBINATION: {best_feature} + {best_model}")
        print(f"   Accuracy: {best_accuracy:.4f}")
        
        if "classification_report_dict" in best_result and "macro avg" in best_result["classification_report_dict"]:
            best_macro_f1 = best_result["classification_report_dict"]["macro avg"]["f1-score"]
            print(f"   Macro F1-Score: {best_macro_f1:.4f}")
else:
    print("No results to display.")

print("\nProcessing completed!")

## Part 2

In [ ]:
# Essential imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif # chi2 is an option for non-negative features
# Import your ML algorithms from Part 1 (ensure these are the same ones)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# Import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# --- Placeholder for your data and models ---
# Ensure these variables are correctly loaded/defined from your Part 0 and Part 1
# Example:
# X_train_dict = {"color": train_color_features, "hog": train_hog_features, ...}
# X_val_dict = {"color": val_color_features, "hog": val_hog_features, ...} # For tuning k or n_components
# X_test_dict = {"color": test_color_features, "hog": test_hog_features, ...}
# y_train, y_val, y_test = your_labels_for_train_val_test_sets

# ML algorithms used in Part 1 (use your actual models and their parameters)
# ml_algorithms_part1 = {
#     "SVM": SVC(kernel='linear', C=1, random_state=42),
#     "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "NaiveBayes": GaussianNB()
# }

# Dictionary to store results from Part 1 (load or ensure this is accessible)
# results_part1 = {
#     "color_SVM": {"accuracy": 0.85, "precision": 0.84, ...},
#     ...
# }

# Dictionary to store results for Part 2
results_part2 = {}

# Helper function to evaluate models
def evaluate_and_store_metrics(model, X_train_proc, y_train_labels, X_test_proc, y_test_labels,
                               feature_set_name, dim_reduction_method, algo_identifier, results_collection):
    print(f"Training {algo_identifier} on {feature_set_name} features processed by {dim_reduction_method}...")
    model.fit(X_train_proc, y_train_labels)
    predictions = model.predict(X_test_proc)

    acc = accuracy_score(y_test_labels, predictions)
    prec = precision_score(y_test_labels, predictions, average='weighted', zero_division=0)
    rec = recall_score(y_test_labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(y_test_labels, predictions, average='weighted', zero_division=0)

    result_key = f"{feature_set_name}_{dim_reduction_method}_{algo_identifier}"
    results_collection[result_key] = {
        "accuracy": acc, "precision": prec, "recall": rec, "f1_score": f1
    }
    print(f"  Metrics for {result_key}: Accuracy={acc:.4f}, F1={f1:.4f}")
    return results_collection

# Get list of feature types you extracted in Part 1
# Make sure X_train_dict, X_val_dict, X_test_dict, y_train, y_val, y_test are defined
# And ml_algorithms_part1 dictionary is defined with your models

feature_extraction_types = list(X_train_dict.keys()) # e.g., ['color_hist', 'hog', 'sift']

for f_type in feature_extraction_types:
    print(f"\n--- Processing Part 2 for feature type: {f_type} ---")

    X_train_current = X_train_dict[f_type]
    # X_val_current = X_val_dict[f_type] # Use for hyperparameter tuning (n_components, k)
    X_test_current = X_test_dict[f_type]

    # 1. PCA Implementation
    print(f"\nApplying PCA to '{f_type}' features...")
    scaler_pca = StandardScaler()
    X_train_scaled = scaler_pca.fit_transform(X_train_current)
    X_test_scaled = scaler_pca.transform(X_test_current)

    # Determine n_components for PCA (e.g., explain 95% variance, or fixed number)
    # pca_explainer = PCA(random_state=42)
    # pca_explainer.fit(X_train_scaled)
    # plt.figure()
    # plt.plot(np.cumsum(pca_explainer.explained_variance_ratio_))
    # plt.xlabel('Number of Components')
    # plt.ylabel('Cumulative Explained Variance')
    # plt.title(f'PCA Explained Variance for {f_type}')
    # plt.grid(True)
    # plt.show()
    # n_pca_components = int(input(f"Enter n_components for PCA on {f_type} based on plot: ")) # Or set automatically
    n_pca_components = 0.95 # Example: Retain 95% of variance

    pca_transformer = PCA(n_components=n_pca_components, random_state=42)
    X_train_pca_transformed = pca_transformer.fit_transform(X_train_scaled)
    X_test_pca_transformed = pca_transformer.transform(X_test_scaled)
    print(f"  Original {f_type} feature dimension: {X_train_scaled.shape[1]}")
    print(f"  PCA-transformed {f_type} feature dimension: {X_train_pca_transformed.shape[1]}")

    for algo_name, base_model in ml_algorithms_part1.items():
        # Re-initialize model to ensure fresh training
        current_ml_model = base_model.__class__(**base_model.get_params())
        results_part2 = evaluate_and_store_metrics(current_ml_model, X_train_pca_transformed, y_train,
                                                 X_test_pca_transformed, y_test,
                                                 f_type, "PCA", algo_name, results_part2)

    # 2. Feature Selection Implementation (e.g., SelectKBest)
    print(f"\nApplying Feature Selection (SelectKBest) to '{f_type}' features...")
    # Note: Scaling can also be applied before SelectKBest if ML algo is sensitive,
    # but f_classif is less dependent on it than PCA. Using X_train_current (unscaled or scaled as per your Part 1).
    # If using chi2, ensure features are non-negative. X_train_scaled might have negative values.
    # For simplicity, let's use X_train_current. If it contains negative values, f_classif is safer than chi2.
    
    num_original_feats = X_train_current.shape[1]
    if num_original_feats <= 1:
        print(f"  Skipping SelectKBest for {f_type} as it has {num_original_feats} feature(s).")
    else:
        # Determine k for SelectKBest (e.g., half the features, or a fixed number like 50/100)
        # k_best_features = max(1, min(100, num_original_feats // 2)) # Example
        # You might want to tune k using X_val_current
        k_best_features = 'all' if num_original_feats < 10 else max(1, num_original_feats // 2) # Adjust k as needed
        if k_best_features != 'all' and k_best_features > num_original_feats:
             k_best_features = num_original_feats


        # Using f_classif as it's generally applicable.
        # If your features are strictly non-negative (e.g. histograms), chi2 is also an option.
        # Ensure X_train_current is suitable for the score_func (e.g. no negative values for chi2)
        # A scaler for feature selection can be added if needed
        # scaler_fs = StandardScaler()
        # X_train_fs_scaled = scaler_fs.fit_transform(X_train_current)
        # X_test_fs_scaled = scaler_fs.transform(X_test_current)

        selector = SelectKBest(score_func=f_classif, k=k_best_features)
        try:
            # Using X_train_current and y_train. If features have negative values and you want to use chi2,
            # you'd need to preprocess them (e.g., MinMaxScaler).
            X_train_selected_feats = selector.fit_transform(X_train_current, y_train)
            X_test_selected_feats = selector.transform(X_test_current)
            print(f"  Original {f_type} feature dimension: {num_original_feats}")
            print(f"  SelectKBest-transformed {f_type} feature dimension: {X_train_selected_feats.shape[1]}")

            for algo_name, base_model in ml_algorithms_part1.items():
                current_ml_model = base_model.__class__(**base_model.get_params())
                results_part2 = evaluate_and_store_metrics(current_ml_model, X_train_selected_feats, y_train,
                                                         X_test_selected_feats, y_test,
                                                         f_type, f"SelectKBest_k{k_best_features}", algo_name, results_part2)
        except ValueError as e:
            print(f"  Error with SelectKBest for {f_type} (k={k_best_features}): {e}. Check for negative inputs if using chi2, or if k is valid.")


# 3. Comparison of All Results
print("\n\n--- Overall Results Comparison ---")

# Convert Part 1 and Part 2 results to DataFrames for easy viewing
results_part1_df = pd.DataFrame.from_dict(results_part1, orient='index')
results_part2_df = pd.DataFrame.from_dict(results_part2, orient='index')

print("\nResults from Part 1 (Original Features):")
print(results_part1_df)

print("\nResults from Part 2 (PCA and Feature Selection):")
print(results_part2_df)

# Combine for a comprehensive table
all_results_list = []
# Part 1
for key, metrics in results_part1.items():
    parts = key.split('_', 1) # Split only on the first underscore
    feature_set = parts[0]
    algorithm = parts[1] if len(parts) > 1 else 'N/A'
    all_results_list.append({'Feature Set': feature_set, 'Method': 'Original', 'Algorithm': algorithm, **metrics})

# Part 2
for key, metrics in results_part2.items():
    parts = key.split('_', 2) # e.g., "color_PCA_SVM" or "hog_SelectKBest_k50_RF"
    feature_set = parts[0]
    method_applied = parts[1]
    algorithm = parts[2]
    if "SelectKBest" in method_applied: # To handle k in the method name if present
         method_parts_further = method_applied.split('_') # e.g. SelectKBest_kVal
         if len(parts) > 2: # if key was color_SelectKBest_k50_SVM
             method_applied = parts[1] # This will be SelectKBest
             algorithm = parts[2]
         else: # if key was color_SelectKBest_SVM (no k in key from evaluate_and_store_metrics)
             algorithm = "N/A" # Or adjust parsing based on your exact key format

    all_results_list.append({'Feature Set': feature_set, 'Method': method_applied, 'Algorithm': algorithm, **metrics})

comparison_df_final = pd.DataFrame(all_results_list)
comparison_df_final = comparison_df_final.set_index(['Feature Set', 'Method', 'Algorithm']).sort_index()

print("\nComprehensive Comparison Table (Accuracy and F1-Score):")
print(comparison_df_final[['accuracy', 'f1_score']])

# --- Add your detailed discussions and interpretations below ---
# For each feature type:
#   - Compare Original vs. PCA vs. SelectKBest for each ML algorithm.
#   - Discuss changes in dimensionality.
#   - Impact on performance (accuracy, precision, recall, F1, training time if measured).
#   - Why might PCA improve/degrade performance? (decorrelation, noise reduction vs. info loss)
#   - Why might Feature Selection improve/degrade performance? (simpler model, reduced overfitting vs. loss of useful info)
#   - Compare PCA vs. Feature Selection directly. Which was better and under what conditions?

In [ ]:
# from sklearn.decomposition import PCA
# import numpy as np
# import matplotlib.pyplot as plt

# pca_temp = PCA().fit(X_train_scaled_for_pca)
# plt.figure(figsize=(8, 5))
# plt.plot(np.cumsum(pca_temp.explained_variance_ratio_))
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.title(f'PCA Explained Variance for {feature_name}')
# plt.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
# plt.legend()
# plt.grid(True, linestyle=':', alpha=0.7)
# plt.show()

### Part 2.3: Applying Feature Selection (SelectKBest) and Evaluating ML Models

In [ ]:

# %% --- Part 2.2: Applying Feature Selection (SelectKBest) and Evaluating ML Models ---
print("\n\n--- Starting Part 2.2: Feature Selection (SelectKBest) ---")

if PART1_FEATURE_TRAIN_DICT_NAME not in globals() or \
   not isinstance(globals()[PART1_FEATURE_TRAIN_DICT_NAME], dict) or \
   'y_train_final' not in globals() or \
   'ml_models' not in globals(): #
    print(f"Error: Part 1 feature sets (e.g., {PART1_FEATURE_TRAIN_DICT_NAME}), labels (y_train_final), or ml_models are not defined/valid. Please run Part 1 first.")
else:
    feature_data_train_source_skb = globals()[PART1_FEATURE_TRAIN_DICT_NAME]
    feature_data_val_source_skb = globals()[PART1_FEATURE_VAL_DICT_NAME]
    feature_data_test_source_skb = globals()[PART1_FEATURE_TEST_DICT_NAME]
    
    for feature_name, X_train_orig in feature_data_train_source_skb.items():
        X_val_orig = feature_data_val_source_skb.get(feature_name)
        X_test_orig = feature_data_test_source_skb.get(feature_name)

        current_y_train = y_train_final
        current_y_val = y_val_final
        current_y_test = y_test_final

        if not isinstance(X_train_orig, np.ndarray) or X_train_orig.size == 0 or \
           (X_train_orig.ndim == 2 and X_train_orig.shape[0] != current_y_train.shape[0]) or \
           (X_train_orig.ndim == 1 and X_train_orig.shape[0] == 0 and current_y_train.shape[0] > 0):
            print(f"\nSkipping SelectKBest for {feature_name}: Training features are empty or mismatched with labels.")
            continue

        run_validation_skb = isinstance(X_val_orig, np.ndarray) and X_val_orig.size > 0 and \
                             isinstance(current_y_val, np.ndarray) and current_y_val.size > 0 and \
                             X_val_orig.shape[0] == current_y_val.shape[0]
        if not run_validation_skb and isinstance(X_val_orig, np.ndarray) and X_val_orig.size > 0:
             print(f"Warning for SelectKBest {feature_name} validation: Label mismatch or empty labels. Validation eval will be skipped.")

        run_test_skb = isinstance(X_test_orig, np.ndarray) and X_test_orig.size > 0 and \
                       isinstance(current_y_test, np.ndarray) and current_y_test.size > 0 and \
                       X_test_orig.shape[0] == current_y_test.shape[0]
        if not run_test_skb and isinstance(X_test_orig, np.ndarray) and X_test_orig.size > 0:
            print(f"Warning for SelectKBest {feature_name} test: Label mismatch or empty labels. Test eval will be skipped.")
            
        print(f"\n===== Applying Feature Selection (SelectKBest) to {feature_name} Features =====")
        print(f"Original Train X shape: {X_train_orig.shape}, y shape: {current_y_train.shape}")

        # 1. Scale original features before feature selection
        scaler_for_skb = StandardScaler()
        X_train_scaled_for_skb = scaler_for_skb.fit_transform(X_train_orig)
        X_val_scaled_for_skb = scaler_for_skb.transform(X_val_orig) if run_validation_skb and X_val_orig.ndim == 2 else np.array([])
        X_test_scaled_for_skb = scaler_for_skb.transform(X_test_orig) if run_test_skb and X_test_orig.ndim == 2 else np.array([])
        
        # 2. Apply SelectKBest
        original_num_features_skb = X_train_scaled_for_skb.shape[1]
        
        if original_num_features_skb == 0:
            print(f"No features to select from for {feature_name} after scaling. Skipping SelectKBest.")
            continue

        # Determine k: min(100, 50% of features, or original_num_features if very few)
        if original_num_features_skb <= 10: 
            k_final_select_skb = original_num_features_skb
        else:
            k_half_select_skb = int(original_num_features_skb * 0.5)
            k_final_select_skb = min(100, k_half_select_skb) 
            if k_final_select_skb == 0 and original_num_features_skb > 0 : k_final_select_skb = 1 # Ensure at least 1 feature

        print(f"Selecting top {k_final_select_skb} features from {feature_name} (original scaled: {original_num_features_skb})...")
        
        try:
            selector_skb = SelectKBest(score_func=f_classif, k=k_final_select_skb)
            X_train_selectkbest = selector_skb.fit_transform(X_train_scaled_for_skb, current_y_train)
            X_val_selectkbest = selector_skb.transform(X_val_scaled_for_skb) if run_validation_skb and X_val_scaled_for_skb.size > 0 else np.array([])
            X_test_selectkbest = selector_skb.transform(X_test_scaled_for_skb) if run_test_skb and X_test_scaled_for_skb.size > 0 else np.array([])
        except ValueError as e_skb: 
             print(f"  SelectKBest Error for {feature_name}: {e_skb}. Possibly k > n_features. Trying k=min(k_final_select_skb, n_features).")
             try:
                 k_fallback_skb_val = min(k_final_select_skb, X_train_scaled_for_skb.shape[1])
                 if k_fallback_skb_val == 0 and X_train_scaled_for_skb.shape[1] > 0: k_fallback_skb_val = 1
                 elif k_fallback_skb_val == 0:
                     print(f"  Cannot select 0 features for {feature_name} (original has {X_train_scaled_for_skb.shape[1]}). Skipping SelectKBest.")
                     continue
                 selector_fallback_skb_inst = SelectKBest(score_func=f_classif, k=k_fallback_skb_val)
                 X_train_selectkbest = selector_fallback_skb_inst.fit_transform(X_train_scaled_for_skb, current_y_train)
                 X_val_selectkbest = selector_fallback_skb_inst.transform(X_val_scaled_for_skb) if run_validation_skb and X_val_scaled_for_skb.size > 0 else np.array([])
                 X_test_selectkbest = selector_fallback_skb_inst.transform(X_test_scaled_for_skb) if run_test_skb and X_test_scaled_for_skb.size > 0 else np.array([])
                 k_final_select_skb = k_fallback_skb_val 
             except Exception as e_fallback_skb_inner:
                 print(f"  SelectKBest Fallback Error for {feature_name}: {e_fallback_skb_inner}. Skipping this feature set for SelectKBest.")
                 continue
        
        print(f"Shape after SelectKBest - Train: {X_train_selectkbest.shape}, Val: {X_val_selectkbest.shape if X_val_selectkbest.size > 0 else 'N/A'}, Test: {X_test_selectkbest.shape if X_test_selectkbest.size > 0 else 'N/A'}")

        # 3. Data was already scaled before selection.

        # 4. Evaluate ML models on selected features
        for model_name, model_instance_orig in ml_models.items(): # Using ml_models from Part 1
            print(f"\n--- Training {model_name} with SelectKBest {feature_name} ---")
            start_time = time.time()
            
            current_model_skb = type(model_instance_orig)(**model_instance_orig.get_params())
            if 'random_state' in current_model_skb.get_params(): current_model_skb.set_params(random_state=42)
            if hasattr(current_model_skb, 'n_jobs') and model_name != "Gaussian Naive Bayes": current_model_skb.set_params(n_jobs=-1)
            
            try:
                current_model_skb.fit(X_train_selectkbest, current_y_train)
                train_time = time.time() - start_time

                val_accuracy_skb = 0.0
                val_report_dict_skb_default = {'macro avg': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0}, 'accuracy': 0.0}
                if run_validation_skb and X_val_selectkbest.size > 0 :
                    y_val_pred_skb = current_model_skb.predict(X_val_selectkbest)
                    val_accuracy_skb = accuracy_score(current_y_val, y_val_pred_skb)
                    labels_val_eval_skb = np.unique(np.concatenate((current_y_val, y_val_pred_skb)))
                    target_names_val_eval_skb = [label_mapping.get(l, str(l)) for l in labels_val_eval_skb] if 'label_mapping' in globals() and label_mapping else [str(l) for l in labels_val_eval_skb]
                    val_report_dict_skb = classification_report(current_y_val, y_val_pred_skb, target_names=target_names_val_eval_skb, labels=labels_val_eval_skb, output_dict=True, zero_division=0)
                    print(f"Validation Accuracy (SelectKBest): {val_accuracy_skb:.4f}")
                else:
                    val_report_dict_skb = val_report_dict_skb_default
                    if run_validation_skb: print(f"Validation evaluation skipped for SelectKBest {feature_name} with {model_name} (empty X_val_selectkbest).")

                test_accuracy_skb = 0.0
                test_report_str_skb = "N/A (Test evaluation skipped)"
                test_report_dict_skb_default = {'macro avg': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0}, 'accuracy': 0.0}
                test_time_skb = 0.0
                if run_test_skb and X_test_selectkbest.size > 0:
                    start_test_time = time.time()
                    y_test_pred_skb = current_model_skb.predict(X_test_selectkbest)
                    test_time_skb = time.time() - start_test_time
                    
                    test_accuracy_skb = accuracy_score(current_y_test, y_test_pred_skb)
                    labels_test_eval_skb = np.unique(np.concatenate((current_y_test, y_test_pred_skb)))
                    target_names_test_eval_skb = [label_mapping.get(l, str(l)) for l in labels_test_eval_skb] if 'label_mapping' in globals() and label_mapping else [str(l) for l in labels_test_eval_skb]

                    test_report_str_skb = classification_report(current_y_test, y_test_pred_skb, target_names=target_names_test_eval_skb, labels=labels_test_eval_skb, zero_division=0)
                    test_report_dict_skb = classification_report(current_y_test, y_test_pred_skb, target_names=target_names_test_eval_skb, labels=labels_test_eval_skb, output_dict=True, zero_division=0)
                    print(f"Test Accuracy (SelectKBest): {test_accuracy_skb:.4f}")
                    print("Test Set Classification Report (SelectKBest):")
                    print(test_report_str_skb)
                else:
                    test_report_dict_skb = test_report_dict_skb_default
                    if run_test_skb: print(f"Test evaluation skipped for SelectKBest {feature_name} with {model_name} (empty X_test_selectkbest).")

                results_part2_selectkbest.append({
                    "Feature Set": f"{feature_name}_SelectKBest_{k_final_select_skb}feat",
                    "ML Model": model_name,
                    "Validation Accuracy": round(val_accuracy_skb, 4),
                    "Test Accuracy": round(test_accuracy_skb, 4),
                    "Test Precision (macro)": round(test_report_dict_skb['macro avg']['precision'], 4),
                    "Test Recall (macro)": round(test_report_dict_skb['macro avg']['recall'], 4),
                    "Test F1-Score (macro)": round(test_report_dict_skb['macro avg']['f1-score'], 4),
                    "Training Time (s)": round(train_time, 2),
                    "Test Time (s)": round(test_time_skb, 2)
                })
            except ValueError as ve_skb_model:
                print(f"ValueError training/evaluating {model_name} with SelectKBest {feature_name}: {ve_skb_model}")
            except Exception as e_skb_model:
                print(f"General error training/evaluating {model_name} with SelectKBest {feature_name}: {e_skb_model}")


### Part 2.4: Displaying Combined Results 

In [ ]:

# %% --- Part 2: Displaying Combined Results ---
print("\n\n--- Summary of Part 2 PCA Results ---")
df_results_part2_pca = pd.DataFrame(results_part2_pca)
if not df_results_part2_pca.empty:
    print(df_results_part2_pca.sort_values(by="Test Accuracy", ascending=False).to_string())
else:
    print("No results to display for Part 2 PCA (results_part2_pca list is empty).")

print("\n\n--- Summary of Part 2 SelectKBest Results ---")
df_results_part2_selectkbest = pd.DataFrame(results_part2_selectkbest)
if not df_results_part2_selectkbest.empty:
    print(df_results_part2_selectkbest.sort_values(by="Test Accuracy", ascending=False).to_string())
else:
    print("No results to display for Part 2 SelectKBest (results_part2_selectkbest list is empty).")

# For a combined comparison, you can concatenate with Part 1 results
# Assuming results_part1_optimized is the LIST of DICTS from your updated Part 1
# Or df_results_part1_optimized is the DATAFRAME from Part 1

df_part1_for_concat_final = pd.DataFrame() # Initialize an empty DataFrame

if 'df_results_part1_optimized' in globals() and isinstance(globals()['df_results_part1_optimized'], pd.DataFrame):
    df_part1_for_concat_final = globals()['df_results_part1_optimized'].copy()
    df_part1_for_concat_final["Processing_Type"] = "Original" # Add a type column
elif 'results_part1_optimized' in globals() and isinstance(globals()['results_part1_optimized'], list):
    df_part1_for_concat_final = pd.DataFrame(globals()['results_part1_optimized'])
    if not df_part1_for_concat_final.empty:
         df_part1_for_concat_final["Processing_Type"] = "Original"
else:
    print("Warning: Part 1 results (results_part1_optimized list or df_results_part1_optimized DataFrame) not found for combined display.")

# Prepare Part 2 DataFrames for concatenation by adding a distinguishing column or modifying "Feature Set"
df_results_part2_pca_display = df_results_part2_pca.copy()
if not df_results_part2_pca_display.empty:
    df_results_part2_pca_display["Processing_Type"] = "PCA"

df_results_part2_skb_display = df_results_part2_selectkbest.copy()
if not df_results_part2_skb_display.empty:
    df_results_part2_skb_display["Processing_Type"] = "SelectKBest"


all_results_dfs_final_combined = []
if not df_part1_for_concat_final.empty:
    all_results_dfs_final_combined.append(df_part1_for_concat_final)
if not df_results_part2_pca_display.empty:
    all_results_dfs_final_combined.append(df_results_part2_pca_display)
if not df_results_part2_skb_display.empty:
    all_results_dfs_final_combined.append(df_results_part2_skb_display)

if all_results_dfs_final_combined:
    df_combined_all_parts = pd.concat(all_results_dfs_final_combined, ignore_index=True)
    print("\n\n--- Combined Summary of Results (Part 1 Original, Part 2 PCA, Part 2 SelectKBest) ---")
    # Sort by Feature Set (original name) and then by Test Accuracy for easier comparison
    # We might need to extract original feature name from the modified "Feature Set" column
    if "Feature Set" in df_combined_all_parts.columns:
        df_combined_all_parts["Original_Feature_Set"] = df_combined_all_parts["Feature Set"].apply(lambda x: x.split('_')[0] if isinstance(x,str) else "Unknown")
        print(df_combined_all_parts.sort_values(by=["Original_Feature_Set", "Test Accuracy"], ascending=[True, False]).to_string())
    else:
        print(df_combined_all_parts.sort_values(by=["Test Accuracy"], ascending=[False]).to_string()) # Fallback sort

else:
    print("No results from any part to combine for the final display.")


# %% [markdown]
# ## Part 2: Comments and Interpretations
#
# * **PCA Application:**
#     * For each feature set (HSV, HoG, Gabor), how many principal components were selected to retain 95% variance? Does this number make sense given the original dimensionality and nature of the features? (e.g., "HoG had an original dimensionality of X, and PCA reduced it to Y components, indicating significant redundancy or correlation in the original HoG features.")
#     * How did classification performance with PCA-transformed features compare to using the original scaled features (from Part 1)? Did PCA improve, degrade, or have a mixed effect on accuracy/F1-score for different ML models? (e.g., "For SVM, PCA on HSV features improved accuracy by Z%, but for Random Forest, it decreased slightly.")
#     * Discuss any significant changes in training/testing times when using PCA. (e.g., "Training time for all models was notably faster with PCA-transformed features due to the lower dimensionality.")
# * **Feature Selection (SelectKBest with f_classif):**
#     * For each feature set, how many features were selected (k)? How does this compare to the original and PCA dimensions?
#     * How did performance with the selected features compare to the original scaled features and the PCA-transformed features? (e.g., "SelectKBest on Gabor features, while reducing features by half, maintained comparable accuracy to the original Gabor set and outperformed PCA for the Logistic Regression model.")
#     * Discuss any significant changes in training/testing times when using feature selection.
# * **Overall Comparison:**
#     * Which approach (original, PCA, or feature selection) yielded the best balance of performance and efficiency (training/test time) for each feature type and each ML algorithm?
#     * Were there any feature types that benefited more from PCA or feature selection than others? Why might this be? (e.g., "Color-based features (HSV) might have seen less improvement from PCA if color channels were already relatively independent, whereas texture features like Gabor might have more correlated components that PCA could effectively reduce.")
#     * Based on your results, what can you conclude about the utility of PCA and feature selection for this specific image classification task using these traditional features? Are they always beneficial? When might one be preferred over the other?
#
# *(Please fill in your detailed comments and interpretations here based on the results you obtain.)*

In [ ]:
if 'df_combined_all_parts_final' in globals() and not df_combined_all_parts.empty:
    plt.figure(figsize=(18, 10))
    # Use 'Original_Feature_Set_Name' for grouping by HSV, HoG, Gabor
    # Use 'Processing_Type' for hue
    sns.barplot(x="ML Model", y="Test Accuracy", hue="Processing_Type", 
                data=df_combined_all_parts, palette="muted",
                # Optional: use facet grid if you want separate plots per Original_Feature_Set_Name
                # col="Original_Feature_Set_Name" 
               )
    plt.title("Overall Model Test Accuracy: Original vs PCA vs SelectKBest", fontsize=18)
    plt.ylabel("Test Accuracy", fontsize=14)
    plt.xlabel("Machine Learning Model", fontsize=14)
    plt.xticks(rotation=45, ha="right")
    plt.legend(title="Processing Type", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("df_combined_all_parts_final DataFrame not found or is empty.")